In [31]:
import os
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [32]:
con_base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256,256,3)
)

In [33]:
con_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [34]:
model = Sequential()

model.add(con_base)
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dense(4,activation='softmax'))

In [35]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 flatten_3 (Flatten)         (None, 32768)             0         
                                                                 
 dense_6 (Dense)             (None, 256)               8388864   
                                                                 
 dense_7 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 23104580 (88.14 MB)
Trainable params: 23104580 (88.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
con_base.trainable = False

In [37]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = './dataset/train/',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
)


groupSize = int(len(train_ds)*0.02)
trainData_0 = train_ds.take(groupSize)
train_ds = trainData_0

validation_ds = keras.utils.image_dataset_from_directory(
    directory = './dataset/validation/',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
)


Found 20000 files belonging to 4 classes.
Found 4000 files belonging to 4 classes.


In [38]:
# Normalize
def process(image,label):
    image = tensorflow.cast(image/255. ,tensorflow.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [39]:
model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [40]:
history = model.fit(train_ds,epochs=10 ,validation_data=validation_ds)

Epoch 1/10


12/12 [==============================] - 629s 57s/step - loss: 4.5861 - accuracy: 0.2630 - val_loss: 1.5058 - val_accuracy: 0.4245
Epoch 2/10
12/12 [==============================] - 4465s 405s/step - loss: 1.2399 - accuracy: 0.4479 - val_loss: 1.3159 - val_accuracy: 0.3758
Epoch 3/10
12/12 [==============================] - ETA: 0s - loss: 0.8878 - accuracy: 0.6042